# 管道模拟

AERO-Suite里的**AERO-S**是固体仿真模拟器，这里是详细的[**AERO-S**教程](https://frg.bitbucket.io/aero-s/index.html)。

本案例用空心圆柱体为例，介绍一系列固体仿真包括：

- 线性静态分析（Linear static analysis）
- 特征值分析 (Eigenvalue analysis)
- 弹性的非线性静态分析 (Nonlinear static analysis for elasticity)
- 弹塑性的非线性静态分析 (Nonlinear static analysis for elasto-plasticity)
- 屈曲分析 (Buckling analysis)
- 非线性动态分析 (Nonlinear dynamic analysis)
- 并行的非线性动态分析 (Parallel nonlinear dynamic analysis)

本问题的设置如下图，我们仅模拟了半个空心圆柱体，固体网格为四边形壳单元，我们考虑了Andes壳单元和 Belytschko-Tsay壳单元： 
<img src="../../Figs/Cylinder.png" width="800" />


本次仿真模拟的文件都在 `Cylinder` 文件夹中，和流体的仿真类似，在这个文件中一般有3个子文件夹：`sources`, `simulations`和`data`。
- `sources` 文件夹一般存放网格相关的源文件
- `simulations` 文件夹一般存放仿真模拟的输入文件，这里的每个子文件夹包含了一个案例
- `data` 文件夹一般存放仿真模拟过程中生成的一些文件。


以下是本次仿真模拟的详细步骤。
在开始本次仿真模拟前，我们需要安装相关的软件，请参加安装教程,比如[如何在北京大学未名一号上安装AERO-Suite](../../../Install/Install_PKU.ipynb)。
如果我们之前已经安装好相关软件，我们可以读取 [.bashrc_frg](../../../Install/bashrc_frg) 刷新当前shell环境，载入各个软件的命令
```shell
source ~/.bashrc_frg 
```


# 线性静态分析（Linear static analysis）

## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Linear_Static/StructureFile](./Linear_Static/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```

然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向），用 `FORCE` 指定顶点或是有限元表面受到的力和力矩，因而也需要指定自由度的编号。这些关键词后面有作用力编号。用关键词 `LOADCASE` 定义不同的受力情况，它们是刚才定义的作用力的一些线性组合。

```shell
PRESSURE 1
1 12202 -676.0
FORCE 2
1027  2 -2000
2450  2  2000
FORCE 3
10524 1 -200000
LOADCASE 1
1 1.0
2 1.0
LOADCASE 2
1 1.0
3 1.0
```

然后指定求解问题的类型，本次仿真是静态分析（STATICS）。需要指定线性方程求解器，和需要分析的受力情况，这里我们会分析两种不同的受力状况。

```shell
STATICS 
SPARSE
CASES 1 2
```


然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "postpro/structure.GDISPLAC.xpost" 1
STRESSVM "postpro/structure.STRESSVM.xpost" 1 upper
STRAINVM "postpro/structure.STRAINVM.xpost" 1 upper
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。

```shell
INCLUDE "../../sources/StructureFile.include.andesshell"
END
```


## 计算及后处理

我们可以在`simulations/Linear_Static`中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

我们感兴趣的一些量，比如固体的位移和Von Mises应力都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Linear_Static` 中，用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.GDISPLAC.xpost postpro/structure.STRESSVM.xpost  postpro/structure.STRAINVM.xpost         
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了两个受力情况下的形变以及上面的Von Mises应力
<img src="../../Figs/Cylinder_Linear_Statics.png" width="800" />

# 特征值分析 (Eigenvalue analysis)

我们考虑描述固体的线性固体力学方程

\begin{align*}
M \ddot{u} + K u = f(t)
\end{align*}

那么这个固体的固有频率 $f_j \quad j = 0, 1,\cdots $， 它们们满足

\begin{align*}
K u_j = 4\pi^2 f_j^2M u_j \rightarrow M \ddot{u} + K u = 0 \qquad u = e^{2\pi i f_j t} u_j
\end{align*}

这块固体一旦开始振动过程，就可以在没有任何外部激励的情况下完全以这个频率进行自由振动，会对固体造成很大的伤害。

## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Eigenvalue/StructureFile](./Eigenvalue/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```
然后指定求解问题的类型，本次仿真是静态分析（STATICS）。需要指定线性方程求解器。

```shell
STATICS 
SPARSE
```

然后用关键词 `EIGEN` 指定是特征值分析，用NEIGPA指定我们要计算前12个特征值。

```shell
EIGEN
NEIGPA 12
NSBSPV 20
TOLEIG 1e-10
*ARPACK LA 3
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GEIGENPA "postpro/structure.GEIGENPA.xpost" 1
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。


```shell
INCLUDE "../../sources/StructureFile.include.andesshell"
END
```


## 计算及后处理

我们可以在`simulations/Eigenvalue`中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

固体结构特征模态都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Eigenvalue` 中，用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.Eigenvalue.xpost 
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了前三个特征模态
<img src="../../Figs/Cylinder_Eigenvalue.png" width="800" />

# 弹性的非线性静态分析 (Nonlinear static analysis for elasticity)

## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Nonlinear_Static_Elasticity/StructureFile](./Nonlinear_Static_Elasticity/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```
然后指定求解问题的类型，本次仿真是静态分析（STATICS）。需要指定线性方程求解器。

```shell
STATICS 
MUMPS
```

然后用关键词 `NONLINEAR` 指定是非线性分析。

```shell
NONLINEAR
```
然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向）。

```shell
PRESSURE
1 12202 -676.0
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "postpro/structure.GDISPLAC.xpost" 1
STRESSVM "postpro/structure.STRESSVM.xpost" 1 upper
STRAINVM "postpro/structure.STRAINVM.xpost" 1 upper
OUTPUT6
GDISPLAC "postpro/structure.GDISPLAC6.xpost" 1
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。

```shell
INCLUDE "../../sources/StructureFile.include.andesshell"
END
```


## 计算及后处理

我们可以在 `simulations/Nonlinear_Static_Elasticity` 中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

固体结构特征模态都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Nonlinear_Static_Elasticity` 中，用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.GDISPLAC.xpost postpro/structure.STRESSVM.xpost  postpro/structure.STRAINVM.xpost         
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了受力情况下的形变以及上面的Von Mises应力
<img src="../../Figs/Cylinder_Nonlinear_Static_Elasticity.png" width="400" />

# 弹塑性的非线性静态分析 (Nonlinear static analysis for elasto-plasticity)

## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Nonlinear_Static_Elastoplasticity/StructureFile](./Nonlinear_Static_Elastoplasticity/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```
然后指定求解问题的类型，本次仿真是静态分析（STATICS）。需要指定线性方程求解器。

```shell
STATICS 
MUMPS
```

然后用关键词 `NONLINEAR` 指定是非线性分析。

```shell
NONLINEAR
```
然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向）。

```shell
PRESSURE
1 12202 -676.0
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "postpro/structure.GDISPLAC.xpost" 1
STRESSVM "postpro/structure.STRESSVM.xpost" 1 upper
STRAINVM "postpro/structure.STRAINVM.xpost" 1 upper
OUTPUT6
GDISPLAC "postpro/structure.GDISPLAC6.xpost" 1
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。值得注意的是[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell)
文件里面并没有指定非线性材料，我们用 `MATUSAGE` 和 `MATLAW` 两个关键词指定了弹塑性材料参数。

```shell
INCLUDE "../../sources/StructureFile.include.andesshell"
MATUSAGE
1 10368 2
MATLAW
2 J2Plasticity 1.014E+7 0.3 2.599E-4 3.909E4 9.3657E4 
END
```


## 计算及后处理

我们可以在`simulations/Nonlinear_Static_Elastoplasticity`中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

固体结构特征模态都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，
我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Nonlinear_Static_Elastoplasticity` 中，
用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.GDISPLAC.xpost postpro/structure.STRESSVM.xpost  postpro/structure.STRAINVM.xpost         
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了受力情况下的形变以及上面的Von Mises应力
<img src="../../Figs/Cylinder_Nonlinear_Static_Elastoplasticity.png" width="400" />

# 屈曲分析 (Buckling analysis)

线性屈曲分析以微小弯曲变形为前提条件，分析理想平衡状态附近的解。
我们考虑描述固体的静态线性固体力学方程

\begin{align*}
K u = f
\end{align*}

以及在荷载$f_G$下发生形变$u_G$, 它对应刚度矩阵$K_G$。那么这个固体在该荷载模式下的屈曲特征值 $\lambda_{G_j}$ 和屈曲模态 $\Delta u_j$

\begin{align*}
(K + \lambda_G K_{G_j}) \Delta u_j = 0 \qquad \forall j=0,\,1,\cdots
\end{align*}

这块固体在该荷载模式下会产生失稳，即荷载不变的情况下发生位移。因而我们需要荷载小于 $\lambda_{G_0}f_G$。


## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Eigenvalue/StructureFile](./Eigenvalue/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```
然后指定求解问题的类型，本次仿真是静态分析（STATICS）。需要指定线性方程求解器。

```shell
STATICS 
SPARSE
```
然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向）。

```shell
PRESSURE
1 12202 -676.0
```

然后用关键词 `BUCKLE` 指定是屈曲分析，用 `GEPS` 指定要考虑几何预应力效应（Geometric Pre-Stress Effects），
这个固体形变状态在 `sources` 文件夹中的[structureFile.gdisplac6Modified](../../sources/structureFile.gdisplac6Modified)文件。关键词是 `IDISP6` 或者 `IDISPLACEMENTS`


```shell
BUCKLE
GEPS
INCLUDE "../../sources/structureFile.gdisplac6Modified"
```

然后用关键词 `EIGEN` 指定特征值的参数，用NEIGPA指定我们要计算前6个特征值。

```shell
EIGEN
NEIGPA 6
NSBSPV 12
TOLEIG 1e-10
SHIFT 1
*ARPACK LA 4
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GEIGENPA "postpro/structure.GEIGENPA.xpost" 1
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.andesshell](../../sources/StructureFile.include.andesshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。


```shell
INCLUDE "../../sources/StructureFile.include.andesshell"
END
```


## 计算及后处理

我们可以在`simulations/Eigenvalue`中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

固体结构屈曲模态都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Buckling` 中，用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.GEIGENPA.xpost 
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了前两个特征模态
<img src="../../Figs/Cylinder_Buckling.png" width="800" />

# 非线性动态分析 (Nonlinear dynamic analysis)


## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Nonlinear_Dynamic_Serial/StructureFile](./Nonlinear_Dynamic_Serial/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```

然后指定求解问题的类型，本次仿真是动态分析（DYNAMICS）。
然后指定时间积分，固体时间积分一般是NEWMARK格式，指定格式参数（MECH），我们采用显示中心差分格式。
以及时间步长（0.8E-07），以及起始时刻。

```shell
DYNAMICS
NEWMARK
MECH 0.0 0.5
TIME 0.0 0.8E-07 0.8e-5
```

然后用关键词 `NONLINEAR` 指定是非线性分析。

```shell
NONLINEAR
```

然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向）。

```shell
PRESSURE
1 12202 -676.0
```

由于我们考虑的是接触碰撞问题，我们用 `CONTACTSURFACES` 指定要曲面之间的接触碰撞法则，
包括曲面编号，法线方向上的容差等。
这些曲面在 [../../sources/surfacetopo.include] 里面用 `SURFACETOPO` 命令定义。

```shell
CONTACTSURFACES
1 1 1 0 0.01
INCLUDE "../../sources/surfacetopo.include"
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "postpro/structure.GDISPLAC.xpost" 1
STRESSVM "postpro/structure.STRESSVM.xpost" 1 upper
STRAINVM "postpro/structure.STRAINVM.xpost" 1 upper
GVELOCIT "postpro/structure.GVELOCIT.2450" 1 2450
OUTPUT6
GDISPLAC "postpro/structure.GDISPLAC6.xpost" 1
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.btshell](../../sources/StructureFile.include.btshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。值得注意的是[StructureFile.include.btshell](../../sources/StructureFile.include.btshell)
文件里面并没有指定非线性材料，我们用 `MATUSAGE` 和 `MATLAW` 两个关键词指定了弹塑性材料参数。

```shell
INCLUDE "../../sources/StructureFile.include.btshell"
MATUSAGE
1 10368 2
MATLAW
2 J2Plasticity 1.014E+7 0.3 2.599E-4 3.909E4 9.3657E4 
END
```


## 计算及后处理

我们可以在 `simulations/Nonlinear_Dynamic_Serial` 中，用以下命令行在1个核上运行**AERO-S*。

```shell
mpirun -n 1 aeros StructureFile |& tee log.out
```

固体结构特征模态都存在了 `postpro` 文件夹。由于这些文件的后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，
我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件, 我们可以在`simulations/Nonlinear_Dynamic_Serial` 中，
用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  postpro/structure.GDISPLAC.xpost postpro/structure.STRESSVM.xpost  postpro/structure.STRAINVM.xpost         
```

我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了受力情况下的形变以及上面的Von Mises应力
<img src="../../Figs/Cylinder_Nonlinear_Dynamic_Serial.png" width="400" />

# 并行的非线性动态分析 (Parallel nonlinear dynamic analysis)

## 前处理

对于固体仿真，很多时候需要使用超算进行并行，我们需要对网格进行分区。
我们使用**AERO-S**把固体网格划分为任意多个区进行并行。
本次仿真模拟我们把网格划分成4个区域，使用4个核进行运算。这可以在 `Cylinder` 文件里用以下命令行完成

```shell
aeros --dec --nsub 4 sources/SowerFile
```
其中 `sources/SowerFile` 是网格文件。

这会在`sources`文件夹中，生成有分区标注的 `SowerFile.optDec` 文件。


之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在 `data` 文件夹中，这可以在 `Cylinder` 文件夹里用以下命令行完成

```shell
sower -struct -mesh sources/SowerFile -dec sources/SowerFile.optDec -cpu 4 -output data/structuremodel -cluster 4
```
这会在 `data` 文件夹中生成以 `structuremodel` 为前缀的文件，包括各区域的网格信息比如 `structuremodel.msh1` ，标注信息比如 `structuremodel.dec1` ，各4个文件，以及CPU和分区的映射 `structuremodel.4cpu` 和各个分区的连接信息 `structuremodel.con`。

这些命令行都可以写在 `Cylinder` 文件夹里的脚本文件里。我们可以用以下命令运行脚本

```shell
bash preprocess.sh
```

## 输入文件
我们的固体部分的仿真输入文件是 [simulations/Nonlinear_Dynamic_Parallel/StructureFile](./Nonlinear_Dynamic_Parallel/StructureFile)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```

然后指定并行处理相关的文件（BINARY），它们是网格分区预处理的生成文件，一般在`data`文件夹里面。
他们包括格点信息 `.msh`、网格分区标注信息 `.dec`, 网格连接信息 `.con`、CPU和分区映射的信息 `.cpu`，
这里也指定了输出是二进制文件。

```shell
BINARY
binaryoutput on 
decomposition "../../data/structuremodel"
cpumap        "../../data/structuremodel.4cpu"
global        "../../data/structuremodel.con"
```

然后指定求解问题的类型，本次仿真是动态分析（DYNAMICS）。
然后指定时间积分，固体时间积分一般是NEWMARK格式，指定格式参数（MECH），我们采用显示中心差分格式。
以及时间步长（0.8E-07），以及起始时刻。

```shell
DYNAMICS
NEWMARK
MECH 0.0 0.5
TIME 0.0 0.8E-07 0.8e-5
```

然后用关键词 `NONLINEAR` 指定是非线性分析。

```shell
NONLINEAR
```

然后用关键词 `PRESSURE` 指定有限元表面受到的压强（外法线方向）。

```shell
PRESSURE
1 12202 -676.0
```

由于我们考虑的是接触碰撞问题，我们用 `CONTACTSURFACES` 指定要曲面之间的接触碰撞法则，
包括曲面编号，法线方向上的容差等。
这些曲面在 `sources`文件夹里的 [surfacetopo.include](../../sources/surfacetopo.include)文件里, 用 `SURFACETOPO` 命令定义。

```shell
CONTACTSURFACES
1 1 1 0 0.01
INCLUDE "../../sources/surfacetopo.include"
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "results/structure.GDISPLAC" 250
STRESSVM "results/structure.STRESSVM" 250 upper
STRAINVM "results/structure.STRAINVM" 250 upper
GVELOCIT "postpro/structure.GVELOCIT.2450" 1 2450
```

最后指定固体网格以及材料参数。这个文件[StructureFile.include.btshell](../../sources/StructureFile.include.btshell) 在 `sources` 文件夹。首先指定的是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号等。
接下来是基本材料属性（MATERIALS）列表。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。值得注意的是[StructureFile.include.btshell](../../sources/StructureFile.include.btshell)
文件里面并没有指定非线性材料，我们用 `MATUSAGE` 和 `MATLAW` 两个关键词指定了弹塑性材料参数。

```shell
INCLUDE "../../sources/StructureFile.include.btshell"
MATUSAGE
1 10368 2
MATLAW
2 J2Plasticity 1.014E+7 0.3 2.599E-4 3.909E4 9.3657E4 
END
```


## 计算及后处理

我们可以在simulations/Nonlinear_Dynamic_Parallel中，用以下命令行在4个核上运行*AERO-S。
```shell
mpirun -n 4 aeros StructureFile |& tee log.out
```

## 结果后处理

由于**AERO-S**的并行，是每个CPU在各自分配到的划分的网格上进行计算，输出结果也是划分网格上的结果，我们需要首先用**sower**把输出结果拼接起来。
我们可以在 `simulations/Nonlinear_Dynamic_Parallel` 中，用以下命令行拼接关于位移、应力、和应变的结果。

```shell
sower -struct -merge -con ../../data/structuremodel.con -mesh ../../data/structuremodel.msh -result results/structure.GDISPLAC 
sower -struct -merge -con ../../data/structuremodel.con -mesh ../../data/structuremodel.msh -result results/structure.STRESSVM 
sower -struct -merge -con ../../data/structuremodel.con -mesh ../../data/structuremodel.msh -result results/structure.STRAINVM 
```

这里的输入包括网格分区的信息，即在`data`文件夹中各个分区的连接信息`structuremodel.con`和各区域的网格信息（比如`structuremodel.msh1`），
以及模拟输出在`results`文件夹中的信息（比如`structure.STRAINVM`）。结果会存放在`results`文件夹里面，文件名会自动加上后缀 `.xpost`。

由于拼接起来的文件是后缀为`.xpost`的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为`.xpost`的文件转化为后缀为`.exo`的文件, 
我们可以在`simulations/Nonlinear_Dynamic_Parallel`中，用以下命令行进行转换

```shell
xp2exo ../../sources/Structure.top  structure.exo  \
             results/structure.GDISPLAC.xpost  \
             results/structure.STRESSVM.xpost  \
             results/structure.STRAINVM.xpost
```

最后，我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `structure.exo` 进行可视化。
这里我们用**paraview**展示了受力情况下的形变以及上面的Von Mises应力
<img src="../../Figs/Cylinder_Nonlinear_Dynamic_Parallel.png" width="400" />